# 01.01 物件的參考、可變性、複製
----
Python 是物件導向的程式語言，這裡提到的細節大多數的時候不影響寫程式，但卻可能是潛在 bug 的來源。

## 變數名稱是標籤
----
物件像是一個箱子，箱子上刻了參考 (id) 與資料型態 (type)，兩者皆不能被改變，箱子內裝了值 (value)。當物件被建立後，指派運算子將變數當作標籤被貼到箱子上，而物件的可變與不可變代表著箱子是否密封 (值是否可修改)，因此 Python 的變數在概念上，就是 Java 的參考變數 (reference variable)。

In [1]:
a = [1, 2, 3, 4]
b = a ; # 指派運算子貼標籤
b[2] = 5
print(a)
print(b)

[1, 2, 5, 4]
[1, 2, 5, 4]


## 集合是存放物件參考的箱子
----
集合 (tuple、list、dict、set) 同樣可視為一個箱子，箱子內存放著物件的參考，參考就像一張寫著物件 id 的紙條，而集合的可變與不可變，代表集合內的參考數量與 id 是否可變，請參考連結文章 [PYTHON OBJECTS: MUTABLE VS. IMMUTABLE](https://codehabitude.com/2013/12/24/python-objects-mutable-vs-immutable/)

如下範例，tuple 不可變，所以集合內的參考數量與 id 不變，但因為 tuple\[-1\] 指向了一個 list，因可以改變 list 內的值。

In [2]:
a = (1, [2])
print(len(a))
print(id(a), id(a[0]), id(a[1]))
print(a, '\n')

a[-1].append(3)
print(len(a))
print(id(a), id(a[0]), id(a[1]))
print(a)

2
1028018081352 1598385632 1028018763400
(1, [2]) 

2
1028018081352 1598385632 1028018763400
(1, [2, 3])


## 集合的複本
----

### 淺複本
----
要複製一個可變集合 (如 list)，最簡單的方式是使用建構式或是在 index 使用 \[**:**\] 或 Slice，產生的新集合雖然是不同的物件，但卻是 **淺複本**，即新集合內的參考與原始集合內的參考相同，因此會發生問題的情況，是在原始集合內有指向可變集合的參考。如下範例，a\[1\].append() 會影響其他淺複本。

In [3]:
a = [1, [2,3], (4,5)]
b = list(a)
c = a[:]
d = a[0:2]

a.append(6)
a[1].append(7)

print(a)
print(b)
print(c)
print(d)

[1, [2, 3, 7], (4, 5), 6]
[1, [2, 3, 7], (4, 5)]
[1, [2, 3, 7], (4, 5)]
[1, [2, 3, 7]]


但對於不可變集合來說，tuple 使用建構式或是在 index 使用 \[**:**\] 不會產生複本，而是回傳同一個物件的參考，如下範例。

In [4]:
t1 = (1,2,3)
t2 = tuple(t1)
print(t2 is t1)

t3 = t1[:]
print(t3 is t1)

True
True


使用 tuple += tuple 時，看起來好像改變了集合，但其實是回傳了新物件，如下範例。

In [5]:
a = [1, [2,3], (4,5)]
print(id(a[2]))

a[2] += (6,7)
print(a)
print(id(a[2]))

1028018360520
[1, [2, 3], (4, 5, 6, 7)]
1028033924216


字串常值在建立時，可能會產生相同的物件，如下範例。

In [6]:
s1 = 'ABC'
s2 = 'ABC'
print(s1 is s2)

True


### 深複本
----
使用 copy.deepcopy() 函式製作 **深複本**，即新集合內指向可變集合的參考，與原始集合內指向可變集合的參考不同，而 copy.copy() 函式是回傳淺複本，如下範例。

In [7]:
import copy

a = [1, [2, 3]]
b = copy.copy(a)
c = copy.deepcopy(a)

print(a)
print(b)
print(c)

print(id(a[0]), id(a[1]))
print(id(b[0]), id(b[1]))
print(id(c[0]), id(c[1]))

[1, [2, 3]]
[1, [2, 3]]
[1, [2, 3]]
1598385632 1028033841480
1598385632 1028033841480
1598385632 1028016559176


## 使用可變物件作函數引數
----
* Parameter - A parameter is an intrinsic property of the procedure, included in its definition
* Argument - The arguments are the values actually supplied to the procedure when it is called

Python 使用 **參考** 來傳遞函數引數，即傳址，在函數內的變數僅為引數物件的標籤，因此當使用可變物件當引數時，可能會造成非使用者預期的結果，如下範例，使用者只想得到新的 list，但輸入的 list 也被改變了。

In [8]:
def get_outputs(inputs):
    inputs.append(2)
    return inputs

inputs = [1, 1]
outputs = get_outputs(inputs)

print(inputs)
print(outputs)

[1, 1, 2]
[1, 1, 2]


如果不想改變可變物件引數，則要在函數內建立引數的複本，如下範例。

In [9]:
def get_outputs(inputs=None):
    outputs = [] if inputs is None else list(inputs) ; # 淺複本，所以 inputs 內不能有可變物件的參照
    outputs.append(2)
    return outputs

inputs = [1, 1]
outputs = get_outputs(inputs)

print(inputs)
print(outputs)

[1, 1]
[1, 1, 2]


### 避免使用可變物件來當成參數預設值
----
當函數被定義時，每一個參數預設值都會被求值 (即通常在模組被載入時)，且預設值會變成函數物件的屬性，所以如果預設值是可變物件，而且你改變它後，這項改變會影響之後每次函數呼叫，如下範例。

In [10]:
def get_outputs(inputs=[]):
    inputs.append(2)
    return inputs

outputs = get_outputs()
print(outputs)

outputs = get_outputs()
print(outputs)

[2]
[2, 2]


## numpy.array 的複本
----
numpy.array (以下用 narray 代替) 也是一種可變集合，但參考數量不可變，並且其內的物件是存放在 data buffer，因此無法使用 id() 檢查 narray 內存的物件，如下範例。

In [11]:
import numpy as np

a = np.array([1,2,3,4])
b = a[0:2]
c = a.reshape(2,2)
d = a.view()
a[0] = 5

print(id(a), id(b), id(c), id(d), '\n')
print(id(a[0]), id(a[1]), id(a[2]), id(a[3]))
print(id(b[0]), id(b[1]))
print(id(c[0,0]), id(c[0,1]), id(c[1,0]), id(c[1,1]))
print(id(d[0]), id(d[1]), id(d[2]), id(d[3]))

1028034077520 1028034080080 1028034078400 1028034078480 

1028288454752 1028288454752 1028288454752 1028288454752
1028288454752 1028288454752
1028288454752 1028288454752 1028288454752 1028288454752
1028288454752 1028288454752 1028288454752 1028288454752


在 narray 的

- index 使用 Slice
- view()
- reshape()

會產生 **淺複本**，官方說法是對資料產生不一樣的 **視圖**，如下範例，可以用 numpy.array.base 檢查 narray 是否為其他 narray 的視圖。

In [12]:
import numpy as np

a = np.array([[1,2],[3,4]])
b = a[0:1]
c = a[0]
d = a.reshape(2,2)
e = a.view()

a[0,0] = 5

print('a =', a)
print('b =', b)
print('c =', c)
print('d =', d)
print('e =', e, '\n')

print('b.base is a :', b.base is a)
print('c.base is a :', c.base is a)
print('d.base is a :', d.base is a)
print('e.base is a :', e.base is a)

a = [[5 2]
 [3 4]]
b = [[5 2]]
c = [5 2]
d = [[5 2]
 [3 4]]
e = [[5 2]
 [3 4]] 

b.base is a : True
c.base is a : True
d.base is a : True
e.base is a : True


在 narray 的

- index 使用 list
- index 使用 tuple
- 建構式
- np.array.copy()
- copy.copy()
- copy.deepcopy()

會產生 **深複本**。

In [13]:
import numpy as np
import copy

a = np.array([[1,2],[3,4]])
b = a[[0, 1]]
c = a[0,1]
d = np.array(a)
e = a.copy()
f = copy.deepcopy(a)

a[0,0] = 5

print('a =', a)
print('b =', b)
print('c =', c)
print('d =', d)
print('e =', e)
print('f =', f, '\n')

print('b.base is a :', b.base is a)
print('c.base is a :', c.base is a)
print('d.base is a :', d.base is a)
print('e.base is a :', e.base is a)
print('f.base is a :', f.base is a)

a = [[5 2]
 [3 4]]
b = [[1 2]
 [3 4]]
c = 2
d = [[1 2]
 [3 4]]
e = [[1 2]
 [3 4]]
f = [[1 2]
 [3 4]] 

b.base is a : False
c.base is a : False
d.base is a : False
e.base is a : False
f.base is a : False


另外，如果在 narray 的 index 填入 tuple 的維度不足，會自動被填入 **:**，被視為使用 Slice，因此會產生 **淺複本**。